## 작업형3

### 문제1. 주어진 조개 데이터 300개 중 앞에서부터 210개는 train 데이터로 만들고, 나머지 90개는 test데이터로 만든다. 모델을 학습(적합)할 때는 train데이터를 사용하고, 예측할 때는 test데이터를 사용한다. 모델은 로지스틱 회귀를 써서 성별(gender)을 예측하되, 패널티는 부과하지 않는다.
- 종속변수: gender(0:암컷, 1:수컷)

### 문제1-1. weight를 독립변수로 gender를 종속변수로 사용하여 로지스틱 회귀 모형을 만들고, weight 변수가 한 단위 증가할 때 수컷일 오즈비 값은? (반올림하여 소수 넷째자리까지 계산)

In [64]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/clam.csv")
train = df.iloc[:210]
test = df.iloc[-90:]

In [65]:
from statsmodels.formula.api import logit
import numpy as np
formula='gender~weight'
model = logit(formula, data=train).fit()
print(model.summary())

round(np.exp(model.params['weight']), 4)

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                 gender   No. Observations:                  210
Model:                          Logit   Df Residuals:                      208
Method:                           MLE   Df Model:                            1
Date:                Fri, 29 Nov 2024   Pseudo R-squ.:                0.003431
Time:                        11:26:34   Log-Likelihood:                -144.91
converged:                       True   LL-Null:                       -145.41
Covariance Type:            nonrobust   LLR p-value:                    0.3178
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3140      0.276     -1.137      0.256      -0.855       0.227
weight         0.0047      0.

1.0047

### 문제1-2. gender를 종속변수로 하고 나머지 변수들(age, length, diameter, height, weight)을 독립변수로 사용하는 로지스틱 회귀 모델을 적합시킨 후, 잔차 이탈도(Residual Deviance)를 계산하시오. (반올림하여 소수 둘째자리까지 계산)

In [66]:
from statsmodels.formula.api import glm
import statsmodels.api as sm
formula='gender~age+length+diameter+height+weight'
model = glm(formula, data=train, family=sm.families.Binomial()).fit()
print(model.summary())
round(286.93, 2)
# 잔차이탈도 Deviance:                       286.93

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 gender   No. Observations:                  210
Model:                            GLM   Df Residuals:                      204
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -143.47
Date:                Fri, 29 Nov 2024   Deviance:                       286.93
Time:                        11:26:34   Pearson chi2:                     210.
No. Iterations:                     4   Pseudo R-squ. (CS):            0.01833
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6701      0.651      1.030      0.3

286.93

### 문제1-3. 독립변수 weight 만 사용해 학습한 로지스틱 회귀모델에서 test데이터의 gender를 예측하고, error rate(오류율)를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [67]:
from statsmodels.formula.api import logit
formula = "gender ~ weight"
model = logit(formula, data=train).fit()
target = test.pop('gender')
pred = model.predict(test) > 0.5
from sklearn.metrics import accuracy_score
accuracy_score(target, pred)

Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4


0.5222222222222223

In [68]:
round(1-accuracy_score(target, pred), 3)

0.478

### 문제2-1. ERP와 가장 상관관계 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [69]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/7_3/system.csv")
df

,ERP,Feature1,Feature2,Feature3,CPU
0,135.281047,235.088034,44.462272,-36.532634,20.492369
1,108.003144,36.571914,46.409312,-21.709347,34.169811
2,119.574760,52.239055,66.494894,-30.590820,67.624248
3,144.817864,196.195890,59.828956,-33.400891,87.923476
4,137.351160,75.175619,59.601973,-26.668085,54.367805
...,...,...,...,...,...
95,114.131463,105.039508,67.053370,-33.807461,33.747738
96,100.210000,141.114900,51.465875,-25.710380,97.049369
97,135.717410,179.363130,58.744305,-24.294491,13.343943
98,102.538242,217.189747,44.008265,-22.667106,9.680395


In [70]:
round(df.corr()['ERP']['Feature1'], 3)

0.434

In [71]:
# 0.434

### 문제2-2. CPU 컬럼이 100미만인 것만 찾아 ERP를 종속 변수로, 나머지 변수들을 독립 변수로 설정해 선형회귀 모델을 만들고 적합한 결정계수를 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [72]:
from statsmodels.formula.api import ols
formula = 'ERP~Feature1+Feature2+Feature3+CPU'
cond = df['CPU']<100
df = df[cond]
model = ols(formula, data=df).fit()
print(model.summary())

# 0.226
print(round(model.rsquared), 3)

                            OLS Regression Results                            
Dep. Variable:                    ERP   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.193
Method:                 Least Squares   F-statistic:                     6.937
Date:                Fri, 29 Nov 2024   Prob (F-statistic):           6.07e-05
Time:                        11:26:34   Log-Likelihood:                -429.44
No. Observations:                 100   AIC:                             868.9
Df Residuals:                      95   BIC:                             881.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     74.4348     15.094      4.932      0.0

### 문제 2-3. 2-2에서 만든 모델에서 독립변수 중 p-value가 가장 높은값을 구하시오. (반올림하여 소수 셋째자리까지 계산)

In [73]:
print(round(model.pvalues.max(), 3))

0.457
